In [1]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

# 데이터 준비

In [2]:
df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# 데이터 탐색

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [4]:
df.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [5]:
# 결측치가 많은 deck열은 삭제
rdf = df.drop("deck", axis = 1)
rdf.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')

In [6]:
# age 열에 결측치가 있는 모든 행을 삭제(177개 행)
rdf = rdf.dropna(subset = ["age"], axis = 0)

In [7]:
len(rdf)

714

In [8]:
# embarked 열의 NaN 값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf["embarked"].value_counts(dropna = True).idxmax()
most_freq

'S'

In [10]:
rdf["embarked"].fillna(most_freq, inplace = True)

In [11]:
rdf.isna().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    2
alive          0
alone          0
dtype: int64

# 분석에 사용할 속성을 선택

In [12]:
ndf = rdf[["survived", "pclass", "sex", "age", "sibsp", "parch", "embarked"]]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [14]:
# 원핫인코딩 - 범주형데이터를 인식할 수 있도록 숫자형으로 변환
onehot_sex = pd.get_dummies(ndf["sex"])

In [15]:
onehot_sex

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
...,...,...
885,1,0
886,0,1
887,1,0
889,0,1


In [16]:
ndf = pd.concat([ndf, onehot_sex], axis = 1)

In [17]:
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male
0,0,3,male,22.0,1,0,S,0,1
1,1,1,female,38.0,1,0,C,1,0
2,1,3,female,26.0,0,0,S,1,0
3,1,1,female,35.0,1,0,S,1,0
4,0,3,male,35.0,0,0,S,0,1


In [18]:
onehot_embarked = pd.get_dummies(ndf["embarked"], prefix = "town")

In [19]:
onehot_embarked.head()

,town_C,town_Q,town_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [20]:
ndf = pd.concat([ndf, onehot_embarked], axis = 1)

In [21]:
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1


In [22]:
ndf.drop(["sex", "embarked"], axis = 1, inplace = True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


# 데이터셋 구분 - 훈련용/테스트용

In [23]:
# 변수 선택
x = ndf.drop("survived", axis = 1) # 독립 변수 x
y = ndf["survived"] # 종속 변수 y

In [24]:
# train data 와 test data 로 구분(7:3 비율)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [25]:
len(x_train), len(x_test)

(499, 215)

# KNN 분류

In [26]:
# KNN 분류 모형 객체 생성 (k = 5로 설정)
knn = KNeighborsClassifier(n_neighbors = 5)

In [27]:
# train data로 모형 학습
knn.fit(x_train, y_train)

KNeighborsClassifier()

In [28]:
# test data로 예측
y_pred = knn.predict(x_test)

In [29]:
pd.DataFrame({"ans" : y_test, "pred" : y_pred})

,ans,pred
423,0,1
177,0,1
305,1,1
292,0,0
889,1,0
...,...,...
203,0,0
499,0,0
628,0,0
879,1,1


In [30]:
# 모형 성능 평가 - Confusion Matrix 계산
knn_matrix = metrics.confusion_matrix(y_test, y_pred)

In [31]:
print(knn_matrix)

[[105  20]
 [ 31  59]]


In [32]:
# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, y_pred)

In [33]:
print(knn_report)

              precision    recall  f1-score   support

           0       0.77      0.84      0.80       125
           1       0.75      0.66      0.70        90

    accuracy                           0.76       215
   macro avg       0.76      0.75      0.75       215
weighted avg       0.76      0.76      0.76       215



- precision(정밀도)
    - 양성으로 예측한 데이터 중 실제로 양성인 데이터의 비율
- recall(재현율, 민감도)
    - 실제 양성인 데이터 중 양성으로 예측한 비율
    
- 코로나 검사키트의 경우
    - 민감도 90% 이상
        - 실제 양성인 사람이 코로나 검사를 하면 양성으로 예측될 확률 90% 이상
    - 특이도 99% 이상
        - 실제 음성인 사람이 코로나 검사를 하면 음성으로 예측될 확률 99% 이상

# 모델 고도화

In [34]:
y.value_counts()

0    424
1    290
Name: survived, dtype: int64

In [35]:
ss = preprocessing.StandardScaler()

In [36]:
scaled_train = ss.fit_transform(x_train)
scaled_test = ss.transform(x_test)

In [37]:
knn = KNeighborsClassifier(n_neighbors=5)

In [38]:
knn.fit(scaled_train, y_train)

KNeighborsClassifier()

In [39]:
y_pred = knn.predict(scaled_test)

In [40]:
pd.DataFrame({"ans" : y_test, "pred" : y_pred})

,ans,pred
423,0,1
177,0,1
305,1,1
292,0,0
889,1,1
...,...,...
203,0,0
499,0,0
628,0,0
879,1,1


In [41]:
knn_matrix = metrics.confusion_matrix(y_test, y_pred)
print(knn_matrix)

[[104  21]
 [ 25  65]]


In [42]:
knn_report = metrics.classification_report(y_test, y_pred)
print(knn_report)

              precision    recall  f1-score   support

           0       0.81      0.83      0.82       125
           1       0.76      0.72      0.74        90

    accuracy                           0.79       215
   macro avg       0.78      0.78      0.78       215
weighted avg       0.79      0.79      0.79       215

